In [1]:
    from textwrap import dedent

In [2]:
    try:
        from .markdown import renderer, CommonMark
        from .environment import environment
    except:
        from markdown import renderer, CommonMark
        from environment import environment
    import IPython
    import types, sys, IPython, asyncio, traceback, ipykernel, traitlets, nbconvert, jinja2, operator, toolz.curried as toolz, tokenize, io, textwrap
    ip = IPython.get_ipython()


    import toolz.curried as toolz

In [3]:
    def expression_tokens(str): 
        """Tokenize expressions.
        """
        try:
            return list(tokenize.generate_tokens(io.StringIO(str).readline))
        except: return []

In [17]:
    import asyncio

In [18]:
    def display_expression_results(ip, expression, result):
        if result['status'] == 'error' or getattr(ip, 'inline', True):
            IPython.display.display(IPython.display.Markdown(
                "`>>> {}`".format(expression)
            ))
            IPython.display.publish_display_data(result.get('data', {}))

In [25]:
    async def evaluate_expression(ip, expressions, globals=None, locals=None):
        globals = globals or ip.user_global_ns
        locals = locals or ip.user_ns

        for expression in split_expression(expressions):
            try: 
                result = ip._format_user_obj(
                    eval(
                        ip.input_transformer_manager.transform_cell(expression), globals, locals))
            except: 
                result = ip._user_obj_error()
                break
                        
        if result['status'] == 'error': 
            result['data'] = {'text/plain': ''.join(''.join(result['traceback']))}
            
        display_expression_results(ip, expressions, result)


In [26]:
    import asyncio

In [27]:
    async def _user_expressions(ip, expressions, globals=None, locals=None):
        futures = [
            evaluate_expression(ip, expression, globals, locals)
            for name, expression in expressions.items()
        ]
        for future in asyncio.as_completed(futures):
            result = await future
            IPython.display.publish_display_data(result.get('data', {}))
        return expressions

In [28]:
    def user_expressions(ip, expressions, globals=None, locals=None):
        asyncio.ensure_future(_user_expressions(ip, expressions, globals, locals))

In [29]:
    def split_expression(str, *expressions):
        """Split an expression on the semi colons."""
        start = 0
        if str.startswith(';'): 
            """This complies with the special IPython ; syntax"""
            return str, 
        for id in toolz.pipe(
            str, expression_tokens, 
            toolz.filter(toolz.compose(
                ';'.__eq__, operator.attrgetter('string')
            )), 
            toolz.map(toolz.compose(
                toolz.second,
                operator.attrgetter('start')
            )),
            list
        ) + [len(str)]:
            expressions += str[start:id].strip(),
            start = id + 1
        return expressions

In [30]:
    def requote(str, token='"""'):
        if token in str: token = "'''"
        return token+str+token